In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import urllib.parse
import pyodbc
import psycopg2 as pg
from datetime import datetime, timedelta


**PROJEÇÕES COBERTURA VACINAL DO ESTADO DE SANTA CATARINA**

In [6]:
#!pip install statsmodels
#!pip install pmdarima
#!pip install psycopg2
#!pip install pyodbc


In [2]:

conexao = pg.connect(database = "******",
                           host="******",
                           user="******",
                           password="******",
                           port="****")


print(conexao.status)


cursor = conexao.cursor()
cursor.execute('SELECT * FROM public.tb_doses_regional_santa_catarina_test')
resultados = cursor.fetchall()


data = pd.DataFrame(resultados, columns=[desc[0] for desc in cursor.description])

1


In [ ]:
DATAJOACABA = data[data['região'] == '316 - AMMOC - JOACABA']
DATAXANXERE = data[data['região'] == '309 - AMAI - XANXERE']
DATAITAJAI = data[data['região'] == '315 - AMFRI - ITAJAI']
DATAMAFRA = data[data['região'] == '319 - AMPLA - MAFRA']
DATACHAPECO = data[data['região'] == '318 - AMOSC - CHAPECO']
DATAFPOLIS = data[data['região'] == '326 - GRANFPOLIS - FPOLIS']
DATATUBARAO = data[data['região'] == '322 - AMUREL - TUBARAO']
DATACRICIUMA = data[data['região'] == '320 - AMREC - CRICIUMA']
DATASAOMIGUEL = data[data['região'] == '313 - AMEOSC - SAO MIGUEL DO OESTE']
DATACONCORDIA = data[data['região'] == '311 - AMAUC - CONCORDIA']
DATALAGES = data[data['região'] == '323 - AMURES - LAGES']
DATABLUMENAU = data[data['região'] == '317 - AMMVI - BLUMENAU']
DATAARARANGUA = data[data['região'] == '314 - AMESC - ARARANGUA']
DATARIOSUL = data[data['região'] == '312 - AMAVI - RIO DO SUL']
DATAVIDEIRA = data[data['região'] == '310 - AMARP - VIDEIRA']
DATAJARAGUA = data[data['região'] == '325 - AMVALI - JARAGUA DO SUL']
DATAJOINVILLE = data[data['região'] == '321 - AMUNESC - JOINVILLE']


def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJOACABA):
    for col in DATAJOACABA.columns.difference(['Data', 'região']):
        if DATAJOACABA[col].dtype == 'O':
            DATAJOACABA[col] = DATAJOACABA[col].str.replace(',', '.').astype(float)
    return DATAJOACABA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJOACABA):
    last_date = DATAJOACABA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJOACABA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJOACABA)

    for series_name in series_names:
        series = DATAJOACABA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJOACABA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJOACABA['Data'] = pd.to_datetime(DATAJOACABA['Data'], format='%Y/%m/%d')
DATAJOACABA.set_index('Data', inplace=True)
DATAJOACABA.sort_index(inplace=True)
DATAJOACABA = preprocess_data(DATAJOACABA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJOACABA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJOACABA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df1 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df1 = pd.concat([DATAJOACABA.reset_index(), forecast_df1], axis=0)
merged_df1.set_index('Data', inplace=True)


####################################################### PROJEÇÃO XANXERE #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAXANXERE):
    for col in DATAXANXERE.columns.difference(['Data', 'região']):
        if DATAXANXERE[col].dtype == 'O':
            DATAXANXERE[col] = DATAXANXERE[col].str.replace(',', '.').astype(float)
    return DATAXANXERE

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAXANXERE):
    last_date = DATAXANXERE.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAXANXERE, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAXANXERE)

    for series_name in series_names:
        series = DATAXANXERE[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAXANXERE[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAXANXERE['Data'] = pd.to_datetime(DATAXANXERE['Data'], format='%Y/%m/%d')
DATAXANXERE.set_index('Data', inplace=True)
DATAXANXERE.sort_index(inplace=True)
DATAXANXERE = preprocess_data(DATAXANXERE)

# Specify the series names you want to forecast
series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAXANXERE, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAXANXERE), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df2 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df2 = pd.concat([DATAXANXERE.reset_index(), forecast_df2], axis=0)
merged_df2.set_index('Data', inplace=True)



####################################################### PROJEÇÃO DATAITAJAI #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAITAJAI):
    for col in DATAITAJAI.columns.difference(['Data', 'região']):
        if DATAITAJAI[col].dtype == 'O':
            DATAITAJAI[col] = DATAITAJAI[col].str.replace(',', '.').astype(float)
    return DATAITAJAI

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAITAJAI):
    last_date = DATAITAJAI.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAITAJAI, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAITAJAI)

    for series_name in series_names:
        series = DATAITAJAI[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAITAJAI[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAITAJAI['Data'] = pd.to_datetime(DATAITAJAI['Data'], format='%Y/%m/%d')
DATAITAJAI.set_index('Data', inplace=True)
DATAITAJAI.sort_index(inplace=True)
DATAITAJAI = preprocess_data(DATAITAJAI)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAITAJAI, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAITAJAI), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df3 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df3 = pd.concat([DATAITAJAI.reset_index(), forecast_df3], axis=0)
merged_df3.set_index('Data', inplace=True)

merged_df3.to_csv('projeção_doses_six_months.csv')

####################################################### PROJEÇÃO DATAMAFRA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAMAFRA):
    for col in DATAMAFRA.columns.difference(['Data', 'região']):
        if DATAMAFRA[col].dtype == 'O':
            DATAMAFRA[col] = DATAMAFRA[col].str.replace(',', '.').astype(float)
    return DATAMAFRA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAMAFRA):
    last_date = DATAMAFRA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAMAFRA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAMAFRA)

    for series_name in series_names:
        series = DATAMAFRA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAMAFRA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAMAFRA['Data'] = pd.to_datetime(DATAMAFRA['Data'], format='%Y/%m/%d')
DATAMAFRA.set_index('Data', inplace=True)
DATAMAFRA.sort_index(inplace=True)
DATAMAFRA = preprocess_data(DATAMAFRA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAMAFRA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAMAFRA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df4 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df4 = pd.concat([DATAMAFRA.reset_index(), forecast_df4], axis=0)
merged_df4.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATACHAPECO #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACHAPECO):
    for col in DATACHAPECO.columns.difference(['Data', 'região']):
        if DATACHAPECO[col].dtype == 'O':
            DATACHAPECO[col] = DATACHAPECO[col].str.replace(',', '.').astype(float)
    return DATACHAPECO

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACHAPECO):
    last_date = DATACHAPECO.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACHAPECO, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACHAPECO)

    for series_name in series_names:
        series = DATACHAPECO[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACHAPECO[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACHAPECO['Data'] = pd.to_datetime(DATACHAPECO['Data'], format='%Y/%m/%d')
DATACHAPECO.set_index('Data', inplace=True)
DATACHAPECO.sort_index(inplace=True)
DATACHAPECO = preprocess_data(DATACHAPECO)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACHAPECO, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACHAPECO), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df5 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df5 = pd.concat([DATACHAPECO.reset_index(), forecast_df5], axis=0)
merged_df5.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATAFPOLIS #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAFPOLIS):
    for col in DATAFPOLIS.columns.difference(['Data', 'região']):
        if DATAFPOLIS[col].dtype == 'O':
            DATAFPOLIS[col] = DATAFPOLIS[col].str.replace(',', '.').astype(float)
    return DATAFPOLIS

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAFPOLIS):
    last_date = DATAFPOLIS.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAFPOLIS, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAFPOLIS)

    for series_name in series_names:
        series = DATAFPOLIS[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAFPOLIS[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAFPOLIS['Data'] = pd.to_datetime(DATAFPOLIS['Data'], format='%Y/%m/%d')
DATAFPOLIS.set_index('Data', inplace=True)
DATAFPOLIS.sort_index(inplace=True)
DATAFPOLIS = preprocess_data(DATAFPOLIS)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAFPOLIS, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAFPOLIS), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df6 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df6 = pd.concat([DATAFPOLIS.reset_index(), forecast_df6], axis=0)
merged_df6.set_index('Data', inplace=True)

###################################################### PROJEÇÃO DATATUBARAO #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATATUBARAO):
    for col in DATATUBARAO.columns.difference(['Data', 'região']):
        if DATATUBARAO[col].dtype == 'O':
            DATATUBARAO[col] = DATATUBARAO[col].str.replace(',', '.').astype(float)
    return DATATUBARAO

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATATUBARAO):
    last_date = DATATUBARAO.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATATUBARAO, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATATUBARAO)

    for series_name in series_names:
        series = DATATUBARAO[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATATUBARAO[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATATUBARAO['Data'] = pd.to_datetime(DATATUBARAO['Data'], format='%Y/%m/%d')
DATATUBARAO.set_index('Data', inplace=True)
DATATUBARAO.sort_index(inplace=True)
DATATUBARAO = preprocess_data(DATATUBARAO)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATATUBARAO, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATATUBARAO), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df7 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df7 = pd.concat([DATATUBARAO.reset_index(), forecast_df7], axis=0)
merged_df7.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATACRICIUMA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACRICIUMA):
    for col in DATACRICIUMA.columns.difference(['Data', 'região']):
        if DATACRICIUMA[col].dtype == 'O':
            DATACRICIUMA[col] = DATACRICIUMA[col].str.replace(',', '.').astype(float)
    return DATACRICIUMA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACRICIUMA):
    last_date = DATACRICIUMA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACRICIUMA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACRICIUMA)

    for series_name in series_names:
        series = DATACRICIUMA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACRICIUMA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACRICIUMA['Data'] = pd.to_datetime(DATACRICIUMA['Data'], format='%Y/%m/%d')
DATACRICIUMA.set_index('Data', inplace=True)
DATACRICIUMA.sort_index(inplace=True)
DATACRICIUMA = preprocess_data(DATACRICIUMA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACRICIUMA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACRICIUMA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df8 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df8 = pd.concat([DATACRICIUMA.reset_index(), forecast_df8], axis=0)
merged_df8.set_index('Data', inplace=True)


####################################################### PROJEÇÃO DATASAOMIGUEL #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATASAOMIGUEL):
    for col in DATASAOMIGUEL.columns.difference(['Data', 'região']):
        if DATASAOMIGUEL[col].dtype == 'O':
            DATASAOMIGUEL[col] = DATASAOMIGUEL[col].str.replace(',', '.').astype(float)
    return DATASAOMIGUEL

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATASAOMIGUEL):
    last_date = DATASAOMIGUEL.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATASAOMIGUEL, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATASAOMIGUEL)

    for series_name in series_names:
        series = DATASAOMIGUEL[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATASAOMIGUEL[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATASAOMIGUEL['Data'] = pd.to_datetime(DATASAOMIGUEL['Data'], format='%Y/%m/%d')
DATASAOMIGUEL.set_index('Data', inplace=True)
DATASAOMIGUEL.sort_index(inplace=True)
DATASAOMIGUEL = preprocess_data(DATASAOMIGUEL)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATASAOMIGUEL, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATASAOMIGUEL), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df10 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df10 = pd.concat([DATASAOMIGUEL.reset_index(), forecast_df10], axis=0)
merged_df10.set_index('Data', inplace=True)


####################################################### PROJEÇÃO DATACONCORDIA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACONCORDIA):
    for col in DATACONCORDIA.columns.difference(['Data', 'região']):
        if DATACONCORDIA[col].dtype == 'O':
            DATACONCORDIA[col] = DATACONCORDIA[col].str.replace(',', '.').astype(float)
    return DATACONCORDIA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACONCORDIA):
    last_date = DATACONCORDIA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACONCORDIA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACONCORDIA)

    for series_name in series_names:
        series = DATACONCORDIA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACONCORDIA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACONCORDIA['Data'] = pd.to_datetime(DATACONCORDIA['Data'], format='%Y/%m/%d')
DATACONCORDIA.set_index('Data', inplace=True)
DATACONCORDIA.sort_index(inplace=True)
DATACONCORDIA = preprocess_data(DATACONCORDIA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACONCORDIA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACONCORDIA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df11 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df11 = pd.concat([DATACONCORDIA.reset_index(), forecast_df11], axis=0)
merged_df11.set_index('Data', inplace=True)

###################################################### PROJEÇÃO DATALAGES #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATALAGES):
    for col in DATALAGES.columns.difference(['Data', 'região']):
        if DATALAGES[col].dtype == 'O':
            DATALAGES[col] = DATALAGES[col].str.replace(',', '.').astype(float)
    return DATALAGES

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATALAGES):
    last_date = DATALAGES.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATALAGES, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATALAGES)

    for series_name in series_names:
        series = DATALAGES[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATALAGES[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATALAGES['Data'] = pd.to_datetime(DATALAGES['Data'], format='%Y/%m/%d')
DATALAGES.set_index('Data', inplace=True)
DATALAGES.sort_index(inplace=True)
DATALAGES = preprocess_data(DATALAGES)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATALAGES, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATALAGES), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df12 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df12 = pd.concat([DATALAGES.reset_index(), forecast_df12], axis=0)
merged_df12.set_index('Data', inplace=True)


###################################################### PROJEÇÃO DATABLUMENAU #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATABLUMENAU):
    for col in DATABLUMENAU.columns.difference(['Data', 'região']):
        if DATABLUMENAU[col].dtype == 'O':
            DATABLUMENAU[col] = DATABLUMENAU[col].str.replace(',', '.').astype(float)
    return DATABLUMENAU

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATABLUMENAU):
    last_date = DATABLUMENAU.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATABLUMENAU, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATABLUMENAU)

    for series_name in series_names:
        series = DATABLUMENAU[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATABLUMENAU[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATABLUMENAU['Data'] = pd.to_datetime(DATABLUMENAU['Data'], format='%Y/%m/%d')
DATABLUMENAU.set_index('Data', inplace=True)
DATABLUMENAU.sort_index(inplace=True)
DATABLUMENAU = preprocess_data(DATABLUMENAU)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATABLUMENAU, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATABLUMENAU), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df13 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df13 = pd.concat([DATABLUMENAU.reset_index(), forecast_df13], axis=0)
merged_df13.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATAARARANGUA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAARARANGUA):
    for col in DATAARARANGUA.columns.difference(['Data', 'região']):
        if DATAARARANGUA[col].dtype == 'O':
            DATAARARANGUA[col] = DATAARARANGUA[col].str.replace(',', '.').astype(float)
    return DATAARARANGUA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAARARANGUA):
    last_date = DATAARARANGUA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAARARANGUA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAARARANGUA)

    for series_name in series_names:
        series = DATAARARANGUA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAARARANGUA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAARARANGUA['Data'] = pd.to_datetime(DATAARARANGUA['Data'], format='%Y/%m/%d')
DATAARARANGUA.set_index('Data', inplace=True)
DATAARARANGUA.sort_index(inplace=True)
DATAARARANGUA = preprocess_data(DATAARARANGUA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAARARANGUA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAARARANGUA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df14 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df14 = pd.concat([DATAARARANGUA.reset_index(), forecast_df14], axis=0)
merged_df14.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATARIOSUL #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATARIOSUL):
    for col in DATARIOSUL.columns.difference(['Data', 'região']):
        if DATARIOSUL[col].dtype == 'O':
            DATARIOSUL[col] = DATARIOSUL[col].str.replace(',', '.').astype(float)
    return DATARIOSUL

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATARIOSUL):
    last_date = DATARIOSUL.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATARIOSUL, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATARIOSUL)

    for series_name in series_names:
        series = DATARIOSUL[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATARIOSUL[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATARIOSUL['Data'] = pd.to_datetime(DATARIOSUL['Data'], format='%Y/%m/%d')
DATARIOSUL.set_index('Data', inplace=True)
DATARIOSUL.sort_index(inplace=True)
DATARIOSUL = preprocess_data(DATARIOSUL)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATARIOSUL, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATARIOSUL), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df15 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df15 = pd.concat([DATARIOSUL.reset_index(), forecast_df15], axis=0)
merged_df15.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATAVIDEIRA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAVIDEIRA):
    for col in DATAVIDEIRA.columns.difference(['Data', 'região']):
        if DATAVIDEIRA[col].dtype == 'O':
            DATAVIDEIRA[col] = DATAVIDEIRA[col].str.replace(',', '.').astype(float)
    return DATAVIDEIRA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAVIDEIRA):
    last_date = DATAVIDEIRA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAVIDEIRA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAVIDEIRA)

    for series_name in series_names:
        series = DATAVIDEIRA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAVIDEIRA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAVIDEIRA['Data'] = pd.to_datetime(DATAVIDEIRA['Data'], format='%Y/%m/%d')
DATAVIDEIRA.set_index('Data', inplace=True)
DATAVIDEIRA.sort_index(inplace=True)
DATAVIDEIRA = preprocess_data(DATAVIDEIRA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAVIDEIRA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAVIDEIRA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df16 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df16 = pd.concat([DATAVIDEIRA.reset_index(), forecast_df16], axis=0)
merged_df16.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATAJARAGUA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJARAGUA):
    for col in DATAJARAGUA.columns.difference(['Data', 'região']):
        if DATAJARAGUA[col].dtype == 'O':
            DATAJARAGUA[col] = DATAJARAGUA[col].str.replace(',', '.').astype(float)
    return DATAJARAGUA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJARAGUA):
    last_date = DATAJARAGUA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJARAGUA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJARAGUA)

    for series_name in series_names:
        series = DATAJARAGUA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJARAGUA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJARAGUA['Data'] = pd.to_datetime(DATAJARAGUA['Data'], format='%Y/%m/%d')
DATAJARAGUA.set_index('Data', inplace=True)
DATAJARAGUA.sort_index(inplace=True)
DATAJARAGUA = preprocess_data(DATAJARAGUA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJARAGUA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJARAGUA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df17 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df17 = pd.concat([DATAJARAGUA.reset_index(), forecast_df17], axis=0)
merged_df17.set_index('Data', inplace=True)

####################################################### PROJEÇÃO DATAJOINVILLE #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJOINVILLE):
    for col in DATAJOINVILLE.columns.difference(['Data', 'região']):
        if DATAJOINVILLE[col].dtype == 'O':
            DATAJOINVILLE[col] = DATAJOINVILLE[col].str.replace(',', '.').astype(float)
    return DATAJOINVILLE

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJOINVILLE):
    last_date = DATAJOINVILLE.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJOINVILLE, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJOINVILLE)

    for series_name in series_names:
        series = DATAJOINVILLE[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJOINVILLE[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJOINVILLE['Data'] = pd.to_datetime(DATAJOINVILLE['Data'], format='%Y/%m/%d')
DATAJOINVILLE.set_index('Data', inplace=True)
DATAJOINVILLE.sort_index(inplace=True)
DATAJOINVILLE = preprocess_data(DATAJOINVILLE)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJOINVILLE, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJOINVILLE), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df18 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df18 = pd.concat([DATAJOINVILLE.reset_index(), forecast_df18], axis=0)
merged_df18.set_index('Data', inplace=True)

In [9]:
###################################################### PROJEÇÃO JOAÇABA #######################################################

def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJOACABA):
    for col in DATAJOACABA.columns.difference(['Data', 'região']):
        if DATAJOACABA[col].dtype == 'O':
            DATAJOACABA[col] = DATAJOACABA[col].str.replace(',', '.').astype(float)
    return DATAJOACABA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJOACABA):
    last_date = DATAJOACABA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJOACABA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJOACABA)

    for series_name in series_names:
        series = DATAJOACABA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJOACABA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJOACABA['Data'] = pd.to_datetime(DATAJOACABA['Data'], format='%Y/%m/%d')
DATAJOACABA.set_index('Data', inplace=True)
DATAJOACABA.sort_index(inplace=True)
DATAJOACABA = preprocess_data(DATAJOACABA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJOACABA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJOACABA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df1 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df1 = pd.concat([DATAJOACABA.reset_index(), forecast_df1], axis=0)
merged_df1.set_index('Data', inplace=True)

C:\Users\jesusda\AppData\Local\Temp\ipykernel_8612\1383333330.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATAJOACABA['Data'] = pd.to_datetime(DATAJOACABA['Data'], format='%Y/%m/%d')
c:\Users\jesusda\Documents\VSCode Projetos\projeto_dash\env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jesusda\Documents\VSCode Projetos\projeto_dash\env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


AttributeError: 'Series' object has no attribute 'append'

In [10]:
####################################################### PROJEÇÃO XANXERE #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAXANXERE):
    for col in DATAXANXERE.columns.difference(['Data', 'região']):
        if DATAXANXERE[col].dtype == 'O':
            DATAXANXERE[col] = DATAXANXERE[col].str.replace(',', '.').astype(float)
    return DATAXANXERE

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAXANXERE):
    last_date = DATAXANXERE.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAXANXERE, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAXANXERE)

    for series_name in series_names:
        series = DATAXANXERE[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAXANXERE[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAXANXERE['Data'] = pd.to_datetime(DATAXANXERE['Data'], format='%Y/%m/%d')
DATAXANXERE.set_index('Data', inplace=True)
DATAXANXERE.sort_index(inplace=True)
DATAXANXERE = preprocess_data(DATAXANXERE)

# Specify the series names you want to forecast
series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAXANXERE, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAXANXERE), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df2 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df2 = pd.concat([DATAXANXERE.reset_index(), forecast_df2], axis=0)
merged_df2.set_index('Data', inplace=True)

C:\Users\jesusda\AppData\Local\Temp\ipykernel_8612\3995921111.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATAXANXERE['Data'] = pd.to_datetime(DATAXANXERE['Data'], format='%Y/%m/%d')
c:\Users\jesusda\Documents\VSCode Projetos\projeto_dash\env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jesusda\Documents\VSCode Projetos\projeto_dash\env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


AttributeError: 'Series' object has no attribute 'append'

In [ ]:
####################################################### PROJEÇÃO DATAITAJAI #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAITAJAI):
    for col in DATAITAJAI.columns.difference(['Data', 'região']):
        if DATAITAJAI[col].dtype == 'O':
            DATAITAJAI[col] = DATAITAJAI[col].str.replace(',', '.').astype(float)
    return DATAITAJAI

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAITAJAI):
    last_date = DATAITAJAI.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAITAJAI, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAITAJAI)

    for series_name in series_names:
        series = DATAITAJAI[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAITAJAI[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAITAJAI['Data'] = pd.to_datetime(DATAITAJAI['Data'], format='%Y/%m/%d')
DATAITAJAI.set_index('Data', inplace=True)
DATAITAJAI.sort_index(inplace=True)
DATAITAJAI = preprocess_data(DATAITAJAI)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAITAJAI, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAITAJAI), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df3 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df3 = pd.concat([DATAITAJAI.reset_index(), forecast_df3], axis=0)
merged_df3.set_index('Data', inplace=True)

merged_df3.to_csv('projeção_doses_six_months.csv')

In [ ]:
####################################################### PROJEÇÃO DATAMAFRA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAMAFRA):
    for col in DATAMAFRA.columns.difference(['Data', 'região']):
        if DATAMAFRA[col].dtype == 'O':
            DATAMAFRA[col] = DATAMAFRA[col].str.replace(',', '.').astype(float)
    return DATAMAFRA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAMAFRA):
    last_date = DATAMAFRA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAMAFRA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAMAFRA)

    for series_name in series_names:
        series = DATAMAFRA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAMAFRA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAMAFRA['Data'] = pd.to_datetime(DATAMAFRA['Data'], format='%Y/%m/%d')
DATAMAFRA.set_index('Data', inplace=True)
DATAMAFRA.sort_index(inplace=True)
DATAMAFRA = preprocess_data(DATAMAFRA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAMAFRA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAMAFRA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df4 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df4 = pd.concat([DATAMAFRA.reset_index(), forecast_df4], axis=0)
merged_df4.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATACHAPECO #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACHAPECO):
    for col in DATACHAPECO.columns.difference(['Data', 'região']):
        if DATACHAPECO[col].dtype == 'O':
            DATACHAPECO[col] = DATACHAPECO[col].str.replace(',', '.').astype(float)
    return DATACHAPECO

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACHAPECO):
    last_date = DATACHAPECO.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACHAPECO, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACHAPECO)

    for series_name in series_names:
        series = DATACHAPECO[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACHAPECO[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACHAPECO['Data'] = pd.to_datetime(DATACHAPECO['Data'], format='%Y/%m/%d')
DATACHAPECO.set_index('Data', inplace=True)
DATACHAPECO.sort_index(inplace=True)
DATACHAPECO = preprocess_data(DATACHAPECO)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACHAPECO, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACHAPECO), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df5 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df5 = pd.concat([DATACHAPECO.reset_index(), forecast_df5], axis=0)
merged_df5.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATAFPOLIS #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAFPOLIS):
    for col in DATAFPOLIS.columns.difference(['Data', 'região']):
        if DATAFPOLIS[col].dtype == 'O':
            DATAFPOLIS[col] = DATAFPOLIS[col].str.replace(',', '.').astype(float)
    return DATAFPOLIS

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAFPOLIS):
    last_date = DATAFPOLIS.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAFPOLIS, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAFPOLIS)

    for series_name in series_names:
        series = DATAFPOLIS[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAFPOLIS[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAFPOLIS['Data'] = pd.to_datetime(DATAFPOLIS['Data'], format='%Y/%m/%d')
DATAFPOLIS.set_index('Data', inplace=True)
DATAFPOLIS.sort_index(inplace=True)
DATAFPOLIS = preprocess_data(DATAFPOLIS)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAFPOLIS, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAFPOLIS), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df6 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df6 = pd.concat([DATAFPOLIS.reset_index(), forecast_df6], axis=0)
merged_df6.set_index('Data', inplace=True)

In [ ]:
###################################################### PROJEÇÃO DATATUBARAO #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATATUBARAO):
    for col in DATATUBARAO.columns.difference(['Data', 'região']):
        if DATATUBARAO[col].dtype == 'O':
            DATATUBARAO[col] = DATATUBARAO[col].str.replace(',', '.').astype(float)
    return DATATUBARAO

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATATUBARAO):
    last_date = DATATUBARAO.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATATUBARAO, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATATUBARAO)

    for series_name in series_names:
        series = DATATUBARAO[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATATUBARAO[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATATUBARAO['Data'] = pd.to_datetime(DATATUBARAO['Data'], format='%Y/%m/%d')
DATATUBARAO.set_index('Data', inplace=True)
DATATUBARAO.sort_index(inplace=True)
DATATUBARAO = preprocess_data(DATATUBARAO)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATATUBARAO, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATATUBARAO), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df7 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df7 = pd.concat([DATATUBARAO.reset_index(), forecast_df7], axis=0)
merged_df7.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATACRICIUMA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACRICIUMA):
    for col in DATACRICIUMA.columns.difference(['Data', 'região']):
        if DATACRICIUMA[col].dtype == 'O':
            DATACRICIUMA[col] = DATACRICIUMA[col].str.replace(',', '.').astype(float)
    return DATACRICIUMA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACRICIUMA):
    last_date = DATACRICIUMA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACRICIUMA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACRICIUMA)

    for series_name in series_names:
        series = DATACRICIUMA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACRICIUMA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACRICIUMA['Data'] = pd.to_datetime(DATACRICIUMA['Data'], format='%Y/%m/%d')
DATACRICIUMA.set_index('Data', inplace=True)
DATACRICIUMA.sort_index(inplace=True)
DATACRICIUMA = preprocess_data(DATACRICIUMA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACRICIUMA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACRICIUMA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df8 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df8 = pd.concat([DATACRICIUMA.reset_index(), forecast_df8], axis=0)
merged_df8.set_index('Data', inplace=True)


In [ ]:
####################################################### PROJEÇÃO DATASAOMIGUEL #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATASAOMIGUEL):
    for col in DATASAOMIGUEL.columns.difference(['Data', 'região']):
        if DATASAOMIGUEL[col].dtype == 'O':
            DATASAOMIGUEL[col] = DATASAOMIGUEL[col].str.replace(',', '.').astype(float)
    return DATASAOMIGUEL

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATASAOMIGUEL):
    last_date = DATASAOMIGUEL.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATASAOMIGUEL, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATASAOMIGUEL)

    for series_name in series_names:
        series = DATASAOMIGUEL[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATASAOMIGUEL[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATASAOMIGUEL['Data'] = pd.to_datetime(DATASAOMIGUEL['Data'], format='%Y/%m/%d')
DATASAOMIGUEL.set_index('Data', inplace=True)
DATASAOMIGUEL.sort_index(inplace=True)
DATASAOMIGUEL = preprocess_data(DATASAOMIGUEL)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATASAOMIGUEL, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATASAOMIGUEL), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df10 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df10 = pd.concat([DATASAOMIGUEL.reset_index(), forecast_df10], axis=0)
merged_df10.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATACONCORDIA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATACONCORDIA):
    for col in DATACONCORDIA.columns.difference(['Data', 'região']):
        if DATACONCORDIA[col].dtype == 'O':
            DATACONCORDIA[col] = DATACONCORDIA[col].str.replace(',', '.').astype(float)
    return DATACONCORDIA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATACONCORDIA):
    last_date = DATACONCORDIA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATACONCORDIA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATACONCORDIA)

    for series_name in series_names:
        series = DATACONCORDIA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATACONCORDIA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATACONCORDIA['Data'] = pd.to_datetime(DATACONCORDIA['Data'], format='%Y/%m/%d')
DATACONCORDIA.set_index('Data', inplace=True)
DATACONCORDIA.sort_index(inplace=True)
DATACONCORDIA = preprocess_data(DATACONCORDIA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATACONCORDIA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATACONCORDIA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df11 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df11 = pd.concat([DATACONCORDIA.reset_index(), forecast_df11], axis=0)
merged_df11.set_index('Data', inplace=True)

In [ ]:
###################################################### PROJEÇÃO DATALAGES #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATALAGES):
    for col in DATALAGES.columns.difference(['Data', 'região']):
        if DATALAGES[col].dtype == 'O':
            DATALAGES[col] = DATALAGES[col].str.replace(',', '.').astype(float)
    return DATALAGES

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATALAGES):
    last_date = DATALAGES.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATALAGES, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATALAGES)

    for series_name in series_names:
        series = DATALAGES[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATALAGES[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATALAGES['Data'] = pd.to_datetime(DATALAGES['Data'], format='%Y/%m/%d')
DATALAGES.set_index('Data', inplace=True)
DATALAGES.sort_index(inplace=True)
DATALAGES = preprocess_data(DATALAGES)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATALAGES, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATALAGES), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df12 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df12 = pd.concat([DATALAGES.reset_index(), forecast_df12], axis=0)
merged_df12.set_index('Data', inplace=True)

In [ ]:
###################################################### PROJEÇÃO DATABLUMENAU #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATABLUMENAU):
    for col in DATABLUMENAU.columns.difference(['Data', 'região']):
        if DATABLUMENAU[col].dtype == 'O':
            DATABLUMENAU[col] = DATABLUMENAU[col].str.replace(',', '.').astype(float)
    return DATABLUMENAU

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATABLUMENAU):
    last_date = DATABLUMENAU.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATABLUMENAU, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATABLUMENAU)

    for series_name in series_names:
        series = DATABLUMENAU[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATABLUMENAU[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATABLUMENAU['Data'] = pd.to_datetime(DATABLUMENAU['Data'], format='%Y/%m/%d')
DATABLUMENAU.set_index('Data', inplace=True)
DATABLUMENAU.sort_index(inplace=True)
DATABLUMENAU = preprocess_data(DATABLUMENAU)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATABLUMENAU, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATABLUMENAU), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df13 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df13 = pd.concat([DATABLUMENAU.reset_index(), forecast_df13], axis=0)
merged_df13.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATAARARANGUA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAARARANGUA):
    for col in DATAARARANGUA.columns.difference(['Data', 'região']):
        if DATAARARANGUA[col].dtype == 'O':
            DATAARARANGUA[col] = DATAARARANGUA[col].str.replace(',', '.').astype(float)
    return DATAARARANGUA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAARARANGUA):
    last_date = DATAARARANGUA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAARARANGUA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAARARANGUA)

    for series_name in series_names:
        series = DATAARARANGUA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAARARANGUA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAARARANGUA['Data'] = pd.to_datetime(DATAARARANGUA['Data'], format='%Y/%m/%d')
DATAARARANGUA.set_index('Data', inplace=True)
DATAARARANGUA.sort_index(inplace=True)
DATAARARANGUA = preprocess_data(DATAARARANGUA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAARARANGUA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAARARANGUA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df14 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df14 = pd.concat([DATAARARANGUA.reset_index(), forecast_df14], axis=0)
merged_df14.set_index('Data', inplace=True)


In [ ]:
####################################################### PROJEÇÃO DATARIOSUL #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATARIOSUL):
    for col in DATARIOSUL.columns.difference(['Data', 'região']):
        if DATARIOSUL[col].dtype == 'O':
            DATARIOSUL[col] = DATARIOSUL[col].str.replace(',', '.').astype(float)
    return DATARIOSUL

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATARIOSUL):
    last_date = DATARIOSUL.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATARIOSUL, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATARIOSUL)

    for series_name in series_names:
        series = DATARIOSUL[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATARIOSUL[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATARIOSUL['Data'] = pd.to_datetime(DATARIOSUL['Data'], format='%Y/%m/%d')
DATARIOSUL.set_index('Data', inplace=True)
DATARIOSUL.sort_index(inplace=True)
DATARIOSUL = preprocess_data(DATARIOSUL)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATARIOSUL, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATARIOSUL), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df15 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df15 = pd.concat([DATARIOSUL.reset_index(), forecast_df15], axis=0)
merged_df15.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATAVIDEIRA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAVIDEIRA):
    for col in DATAVIDEIRA.columns.difference(['Data', 'região']):
        if DATAVIDEIRA[col].dtype == 'O':
            DATAVIDEIRA[col] = DATAVIDEIRA[col].str.replace(',', '.').astype(float)
    return DATAVIDEIRA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAVIDEIRA):
    last_date = DATAVIDEIRA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAVIDEIRA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAVIDEIRA)

    for series_name in series_names:
        series = DATAVIDEIRA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAVIDEIRA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAVIDEIRA['Data'] = pd.to_datetime(DATAVIDEIRA['Data'], format='%Y/%m/%d')
DATAVIDEIRA.set_index('Data', inplace=True)
DATAVIDEIRA.sort_index(inplace=True)
DATAVIDEIRA = preprocess_data(DATAVIDEIRA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAVIDEIRA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAVIDEIRA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df16 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df16 = pd.concat([DATAVIDEIRA.reset_index(), forecast_df16], axis=0)
merged_df16.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATAJARAGUA #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJARAGUA):
    for col in DATAJARAGUA.columns.difference(['Data', 'região']):
        if DATAJARAGUA[col].dtype == 'O':
            DATAJARAGUA[col] = DATAJARAGUA[col].str.replace(',', '.').astype(float)
    return DATAJARAGUA

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJARAGUA):
    last_date = DATAJARAGUA.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJARAGUA, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJARAGUA)

    for series_name in series_names:
        series = DATAJARAGUA[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJARAGUA[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJARAGUA['Data'] = pd.to_datetime(DATAJARAGUA['Data'], format='%Y/%m/%d')
DATAJARAGUA.set_index('Data', inplace=True)
DATAJARAGUA.sort_index(inplace=True)
DATAJARAGUA = preprocess_data(DATAJARAGUA)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJARAGUA, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJARAGUA), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df17 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df17 = pd.concat([DATAJARAGUA.reset_index(), forecast_df17], axis=0)
merged_df17.set_index('Data', inplace=True)

In [ ]:
####################################################### PROJEÇÃO DATAJOINVILLE #######################################################
def fit_auto_arima(series, seasonal=True, max_order=None, trace=False, error_action='ignore',
                    alpha=0.05, stepwise=True, suppress_warnings=True):
    model = auto_arima(series, seasonal=seasonal, stepwise=stepwise, suppress_warnings=suppress_warnings,
                       trace=trace, error_action=error_action, max_order=max_order,
                       alpha=alpha)
    order = model.order
    model = SARIMAX(series, order=order)
    results = model.fit()
    return results

def preprocess_data(DATAJOINVILLE):
    for col in DATAJOINVILLE.columns.difference(['Data', 'região']):
        if DATAJOINVILLE[col].dtype == 'O':
            DATAJOINVILLE[col] = DATAJOINVILLE[col].str.replace(',', '.').astype(float)
    return DATAJOINVILLE

def calculate_average(series):
    last_value = series.iloc[-1]
    second_last_value = series.iloc[-2]
    return (last_value + second_last_value) / 2

def get_next_month_date(DATAJOINVILLE):
    last_date = DATAJOINVILLE.index[-1]
    next_month_date = last_date + pd.DateOffset(months=1)
    return next_month_date

def project_next_months(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), num_months=6):
    current_series = series.copy()

    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    next_date = datetime(current_year, current_month, 1) + pd.DateOffset(months=1)

    projected_values = []

    for _ in range(num_months):
        model = SARIMAX(current_series, order=order, seasonal_order=seasonal_order)
        model_fit = model.fit(disp=False)
        forecast = model_fit.get_forecast(steps=1)

        if forecast.predicted_mean.empty:
            forecast_value = calculate_average(current_series)
        else:
            forecast_value = round(forecast.predicted_mean.values[0], 0)

        if forecast_value == 0 or (current_series == 0).sum() > len(current_series) / 2:
            forecast_value = calculate_average(current_series)

        projected_values.append(forecast_value)
        current_series = current_series.append(pd.Series([forecast_value], index=[next_date]))
        next_date += pd.DateOffset(months=1)

    next_month_series = series.append(pd.Series(projected_values, index=pd.date_range(start=next_date, periods=num_months, freq='M')))

    return next_month_series, projected_values

def make_forecasts_with_projection(DATAJOINVILLE, series_names, max_order=True, trace=True, error_action='ignore',
                                    alpha=0.05, stepwise=True, suppress_warnings=True, num_months=6):
    forecast_dict = {}
    next_month_date = get_next_month_date(DATAJOINVILLE)

    for series_name in series_names:
        series = DATAJOINVILLE[series_name]

        if (series == 0).any():
            series = series.replace(0, np.nan).interpolate(method='linear')

        projected_series, forecast_values = project_next_months(series, num_months=num_months)
        DATAJOINVILLE[series_name] = projected_series

        results = fit_auto_arima(projected_series, seasonal=True, max_order=max_order, trace=trace,
                                  error_action=error_action, alpha=alpha, stepwise=stepwise,
                                  suppress_warnings=suppress_warnings)

        forecast = results.get_forecast(steps=num_months)

        forecast_dict[series_name] = forecast_values

    return forecast_dict

DATAJOINVILLE['Data'] = pd.to_datetime(DATAJOINVILLE['Data'], format='%Y/%m/%d')
DATAJOINVILLE.set_index('Data', inplace=True)
DATAJOINVILLE.sort_index(inplace=True)
DATAJOINVILLE = preprocess_data(DATAJOINVILLE)

series_names = ['DTP - 01 ano(1º REF)', 'Poliomielite(VOP/VIP)(1ºREF)', 'FA(< 1 ano)', 'varicela', 'Tríplice Viral - D1', 
                 'Rotavírus Humano', 'Pneumocóccica(1 ano)', 'Menigocócica Conj.C(< 1 ano)', 'Pentavalente (< 1 ano)', 'Tríplice Viral - D2', 
                 'Hepatite A', 'Pneumocóccica(<1 ano)', 'bcg','Menigocócica Conj.C(1 ano)', 'Poliomielite(< 1 ano)']

max_order = (3, 3, 3)
trace = True
error_action = 'ignore'
alpha = 0.05
stepwise = True
suppress_warnings = True

forecast_dict_next_six_months = make_forecasts_with_projection(DATAJOINVILLE, series_names, max_order=max_order, trace=trace,
                                                          error_action=error_action, alpha=alpha, stepwise=stepwise,
                                                          suppress_warnings=suppress_warnings, num_months=6)

forecast_next_six_months_df = pd.DataFrame(forecast_dict_next_six_months, index=pd.date_range(start=get_next_month_date(DATAJOINVILLE), periods=6, freq='M'))
forecast_next_six_months_df.index.name = 'Data'
forecast_next_six_months_df = forecast_next_six_months_df.reset_index()

def remove_decimal_and_convert_to_int(value):
    if pd.notna(value):
        if isinstance(value, (float, float)):
            return int(value)
        elif isinstance(value, str):
            parts = value.split('.')
            if len(parts) > 0:
                return int(parts[0])
    return value

forecast_df18 = forecast_next_six_months_df.applymap(remove_decimal_and_convert_to_int)
merged_df18 = pd.concat([DATAJOINVILLE.reset_index(), forecast_df18], axis=0)
merged_df18.set_index('Data', inplace=True)

In [ ]:
                           ############################## MERGE DOS RESULTADOS ##############################


forecast_dfs = [forecast_df1, forecast_df2, forecast_df3, forecast_df4, forecast_df5, forecast_df6, forecast_df7, forecast_df8, forecast_df10, forecast_df11, forecast_df12, forecast_df13, forecast_df14, forecast_df15, forecast_df16, forecast_df17, forecast_df18]

resultado = pd.concat([forecast_df1, forecast_df2], axis=0)
resultado1 = pd.concat([forecast_df3, resultado], axis=0)
resultado2 = pd.concat([forecast_df4, resultado1], axis=0)
resultado3 = pd.concat([forecast_df5, resultado2], axis=0)
resultado4 = pd.concat([forecast_df6, resultado3], axis=0)
resultado5 = pd.concat([forecast_df7, resultado4], axis=0)
resultado6 = pd.concat([forecast_df8, resultado5], axis=0)
resultado7 = pd.concat([forecast_df10, resultado6], axis=0)
resultado8 = pd.concat([forecast_df11, resultado7], axis=0)
resultado9 = pd.concat([forecast_df12, resultado8], axis=0)
resultado10 = pd.concat([forecast_df13, resultado9], axis=0)
resultado11 = pd.concat([forecast_df14, resultado10], axis=0)
resultado12 = pd.concat([forecast_df15, resultado11], axis=0)
resultado13 = pd.concat([forecast_df16, resultado12], axis=0)
resultado14 = pd.concat([forecast_df17, resultado13], axis=0)
resultado_final = pd.concat([forecast_df18, resultado14], axis=0)
print(resultado_final)

In [ ]:
resultado_final.reset_index(inplace=True)

resultado_final['mes'] = pd.to_datetime(resultado_final['Data'].astype(str)).dt.strftime('%B') #adicionar o nome do mes dentro da coluna
resultado_final['ano'] = pd.to_datetime(resultado_final['Data'].astype(str)).dt.strftime('%Y') #adicionar o nome do ano dentro da coluna

meses_dict = {  
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}

resultado_final['mes'] = resultado_final['mes'].map(meses_dict)

data_atual = datetime.now()
mes_atual = data_atual.month
ano_atual = data_atual.year
resultado_final['projetada'] = resultado_final['Data'].apply(lambda x: (x.year, x.month) <= (ano_atual, mes_atual))
resultado_final['projetada'] = ~resultado_final['projetada'] #inverter o true e false


print(resultado_final.tail(5))

resultado_final.to_excel('last_result.xlsx')

In [ ]:
for index, row in resultado_final.iterrows():
    cursor.execute(
        'INSERT INTO imunizacao.tb_doses_regional_santa_catarina'
        '("Data", "região", "DTP - 01 ano(1º REF)", "Poliomielite(VOP/VIP)(1ºREF)", "FA(< 1 ano)", "varicela", '
        '"Tríplice Viral - D1", "Rotavírus Humano", "Pneumocóccica(1 ano)", "Menigocócica Conj.C(< 1 ano)", '
        '"Pentavalente (< 1 ano)", "Tríplice Viral - D2", "Hepatite A", "Pneumocóccica(<1 ano)", "bcg", '
        '"Menigocócica Conj.C(1 ano)", "Poliomielite(< 1 ano)", "mes", "ano", "projetada") '
        'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);',
        (
            row["Data"],
            row["região"],
            row["DTP - 01 ano(1º REF)"],
            row["Poliomielite(VOP/VIP)(1ºREF)"],
            row["FA(< 1 ano)"],
            row["varicela"],
            row["Tríplice Viral - D1"],
            row["Rotavírus Humano"],
            row["Pneumocóccica(1 ano)"],
            row["Menigocócica Conj.C(< 1 ano)"],
            row["Pentavalente (< 1 ano)"],
            row["Tríplice Viral - D2"],
            row["Hepatite A"],
            row["Pneumocóccica(<1 ano)"],
            row["bcg"],
            row["Menigocócica Conj.C(1 ano)"],
            row["Poliomielite(< 1 ano)"],
            row["mes"],
            row["ano"],
            row["projetada"]
        )
    )

conexao.commit()
cursor.close()
conexao.close()

resultado_final